# Sock Market Screener

In [3]:
from tradingview_screener import Query, col
import pandas as pd

tickers = (Query()
 .select('name', 'close', 'volume', 'relative_volume_10d_calc')
 .where(
     col('type') == 'stock',
     col('exchange') == 'NASDAQ',
     col('close').between(25, 250),
     col('market_cap_basic') > 1_000_000_000,
     
 )
 .order_by('volume', ascending=False)
 .offset(5)
 .limit(25)
 .get_scanner_data())

print(tickers) 



(579,           ticker   name    close    volume  relative_volume_10d_calc
0    NASDAQ:HOOD   HOOD  104.850  35636476                  0.753353
1    NASDAQ:SMCI   SMCI   54.470  34307948                  1.012580
2   NASDAQ:CMCSA  CMCSA   33.680  32924574                  1.950708
3     NASDAQ:CSX    CSX   35.760  31707887                  1.409535
4    NASDAQ:GOOG   GOOG  194.080  31101570                  1.182955
5    NASDAQ:AMZN   AMZN  231.440  28711843                  0.756915
6    NASDAQ:ASTS   ASTS   54.335  25294530                  2.125479
7     NASDAQ:TRI    TRI  203.450  23377477                 11.587387
8      NASDAQ:MU     MU  111.260  16372323                  0.666164
9    NASDAQ:RKLB   RKLB   47.430  14517933                  0.544799
10   NASDAQ:CSCO   CSCO   68.690  11590008                  0.759631
11   NASDAQ:CRWV   CRWV  115.620  10297253                  0.892544
12   NASDAQ:MRVL   MRVL   74.210   9830297                  0.673600
13    NASDAQ:KDP    KDP   33

In [ ]:
from tradingview_screener import Query, col
import pandas as pd

tickers = (Query()
 .select('name', 'close', 'volume', 'relative_volume_10d_calc')
 .where(
     col('type') == 'stock',
     col('exchange') == 'NASDAQ',
     col('close').between(25, 250),
     col('market_cap_basic') > 1_000_000_000,
     
 )
 .order_by('volume', ascending=False)
 .offset(5)
 .limit(25)
 .get_scanner_data())

print(tickers) 

